# Infoli Task Inspector

In [19]:
import import_ipynb
from infoliLauncher import infoliTask, pklFile
import pickle as pkl

This notebook gives the possibility to inspect the list of tasks saved in the file defined by `pklFile` while the `infoliLauncher` notebook is still running simulations.

In [20]:
with open(pklFile, "rb") as f:
    listOfTasks = pkl.load(f)

In [21]:
listOfTasks[1].getData()

{'parameters': {'simTime': 0.003,
  'connectivity': 0.6993484319038994,
  'neurons': 8251,
  'memory': 4096,
  'CPU': 4,
  'timeout': 14400},
 'results': {'hasExecuted': True,
  'setupTime': 0.5,
  'execTime': 1629.7,
  'startTime': 1582721735.810962,
  'endTime': 1582723366.8867707,
  'measureTime': 1631.075808763504,
  'hasEndedInTimeout': False,
  'hasEndedInError': False}}